<center><h1>Data Generation Experiments</h1></center>

<img src="logo_autodata.png" alt="drawing" width="200"/>

In [6]:
import autopandas as apd
import numpy as np
from sklearn.neural_network import MLPClassifier
from IPython.display import display
%reload_ext autoreload
%autoreload 2

In [2]:
def load_dataset(input_file, test_file=None):
    if test_file is None:
        data = apd.read_csv(input_file)
    else:
        train = apd.read_csv(input_file)
        test = apd.read_csv(test_file)
        data = apd.from_train_test(train, test)
    # processing
    data = data.imputation()
    data = data.encoding()
    data = data.normalization()
    # TMP reduce size
    #data = data[:min(len(data), 200)]
    return data

In [3]:
def describe_dataset(data):
    # TODO
    # read data
    pass
    # cat/num
    # descriptors
    # PCA, TSNE, LDA, y dist

In [7]:
def run_experiment(datasets_list, generators_list):
    # Generate data and compute metrics
    for dataset in datasets_list:
        data = load_dataset(dataset)
        print('\n')
        print(dataset)
        print('=================================================')
        describe_dataset(data)
        
        # to store results
        names = ['Generator']
        nnaas = ['NNAA']
        discriminants = ['Adversarial score']
        discrepancies = ['NN discrepancy']
        
        for generator in generators_list:
            # init generator
            if generator is None:
                generator = apd.generators.Copycat
            generator = generator()
            
            names.append(generator.__class__.__name__)
            
            # training
            generator.fit(data)
            gen_data = generator.sample(n=data.shape[0])
            
            # metrics
            nnaas.append(data.distance(gen_data, method='nnaa'))
            discriminants.append(data.distance(gen_data, method='discriminant')) # and with another model?
            discrepancies.append(data.distance(gen_data))
            # task score
            
            # plots
            # overlaying PCA and all
            #data.compare_marginals(gen_data, method='all')
            
        # Store results into a table
        results = np.array([names, nnaas, discriminants, discrepancies]).T
        results = apd.AutoData(results[1:,:], columns=results[0,:])
        display(results)

In [9]:
path = 'autopandas/data/'
datasets = [path+x for x in ['iris.csv', 'wine.csv']] #, 'diabetes.csv', 'seeds.csv', 'adult.csv',
                             #'mushrooms.csv', 'wine.csv', 'squares.csv']]
# train/test boston, titanic
generators = [apd.generators.Copycat,
              apd.generators.KDE,
              apd.generators.ANM,
              apd.generators.Copula,
              apd.generators.GMM] # VAE, KDE, +
                
run_experiment(datasets, generators)



autopandas/data/iris.csv


/usr/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,Generator,NNAA,Adversarial score,NN discrepancy
0,Copycat,0.0,0.5,0
1,KDE,0.6233333333333333,0.6222222222222222,0.6866666666666665
2,ANM,0.4966666666666667,0.4777777777777778,0.07333333333333325
3,Copula,0.7133333333333334,0.4444444444444444,0.6933333333333334
4,GMM,0.6933333333333334,0.4,0.44666666666666677




autopandas/data/wine.csv


/usr/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


KeyboardInterrupt: 